In [1]:
#importing all the required libaries 
import pandas as pd
import os
import numpy as np
import shutil
import glob

### Enter the path to weather_station full_dataset

In [2]:
weather_data_file_path= r'C:\Users\raramachandr\OneDrive - NVIDIA Corporation\Documents\Personal\Masters_files_desktop\project\New_files\smokeynet-engineering-main1\data\raw\weather_new'

### Import the camera_to_weather_station mapping file

In [3]:
camera_station_mapping = pd.read_csv("../../data/processed/camera_station_mappings.csv")
camera_station_mapping.head()

,camera_id,image_id,stid,shortname,distance_m,distance_mi,is_in_direction,rn
0,hpwren0_unknown direction,tje-1-mobo-c,BFDSD,SDGE,2362.963219,1.468277,NaN,1
1,hpwren0_unknown direction,tje-1-mobo-c,CVXSD,SDGE,8161.908209,5.071575,NaN,2
2,hpwren0_unknown direction,tje-1-mobo-c,DJZSD,SDGE,10216.081526,6.347979,NaN,3
3,hpwren1_north,bm-n-mobo-c,MGDSD,SDGE,4635.802946,2.880554,True,1
4,hpwren1_north,bm-n-mobo-c,HP004,HPWREN,5385.340294,3.346295,True,2


In [4]:
camera_station_mapping.shape

(391, 8)

### Import the camera ID to camera_name file (provided in Fiblib_files)

In [5]:
camera_station_mapping2 = pd.read_csv("../../data/raw/camera_id_mapping.csv")
camera_station_mapping2.head()

,Camera_ID,camera_name
0,hpwren0_unknown direction,tje-1-mobo-c
1,hpwren1_east,bm-e-mobo-c
2,hpwren1_north,bm-n-mobo-c
3,hpwren1_north,bm-n-mobo
4,hpwren1_south,bm-s-mobo-c


### Merge both the mappnig files

In [6]:
camera_station_mapping3 = pd.merge(camera_station_mapping,camera_station_mapping2, how='left', left_on ="camera_id", right_on = "Camera_ID")

### Calculate inverse distance of every weather station so that closest station get higest weight

In [7]:
# Find max distance and subract from the distance column (to get inverse of distance)
max_distance_calculated = camera_station_mapping3.distance_mi.max()+1
camera_station_mapping3["distance_mi_inverse"] = max_distance_calculated - camera_station_mapping3["distance_mi"]
#camera_station_mapping3.head()

### Split the weather data into multiple small chuck of file.

In [9]:
class FileSettings(object):
    def __init__(self, file_name, row_size=1000000):
        self.file_name = file_name
        self.row_size = row_size


class FileSplitter(object):

    def __init__(self, file_settings):
        self.file_settings = file_settings

        if type(self.file_settings).__name__ != "FileSettings":
            raise Exception("Please pass correct instance ")

        self.df = pd.read_csv(self.file_settings.file_name,
                              chunksize=self.file_settings.row_size)

    def run(self, directory="split_files"):
        try:os.makedirs(directory)
        except Exception as e:pass

        counter = 0

        while True:
            try:
                file_name = "{}/{}_{}_row_{}.csv".format(
                    directory,  self.file_settings.file_name.split(".")[0], counter, self.file_settings.row_size
                ) 
                df = next(self.df).to_csv(file_name,index =False)
                counter = counter + 1
            except StopIteration:
                break
            except Exception as e:
                print("Error:",e)
                break

        return True

In [10]:
### Run the filter spliter for the 3 group of weather Stations
os.chdir(weather_data_file_path)

helper =  FileSplitter(FileSettings(
    file_name='weather_HPWREN.csv',
    row_size=2000000
))
helper.run()

helper =  FileSplitter(FileSettings(
    file_name='weather_SC-EDISON.csv',
    row_size=2000000
))
helper.run()

helper =  FileSplitter(FileSettings(
    file_name='weather_SDGE.csv',
    row_size=2000000
))
helper.run()

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (2,3,4,5,6,7) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


True

### Create the folder inside the temp directory weather_data by weatherstation directory

In [11]:
weather_data_file_path_split = weather_data_file_path + r'\split_files'
os.chdir(weather_data_file_path_split)
shutil.rmtree(weather_data_file_path_split +r'\ByWeatherStation',ignore_errors=True)

weather_split_file_list =  glob.glob('*.csv')
try:os.makedirs("ByWeatherStation")
except Exception as e:pass

In [12]:
weather_split_file_list

['weather_HPWREN_0_row_2000000.csv',
 'weather_HPWREN_1_row_2000000.csv',
 'weather_SC-EDISON_0_row_2000000.csv',
 'weather_SC-EDISON_1_row_2000000.csv',
 'weather_SC-EDISON_2_row_2000000.csv',
 'weather_SC-EDISON_3_row_2000000.csv',
 'weather_SDGE_0_row_2000000.csv',
 'weather_SDGE_10_row_2000000.csv',
 'weather_SDGE_11_row_2000000.csv',
 'weather_SDGE_1_row_2000000.csv',
 'weather_SDGE_2_row_2000000.csv',
 'weather_SDGE_3_row_2000000.csv',
 'weather_SDGE_4_row_2000000.csv',
 'weather_SDGE_5_row_2000000.csv',
 'weather_SDGE_6_row_2000000.csv',
 'weather_SDGE_7_row_2000000.csv',
 'weather_SDGE_8_row_2000000.csv',
 'weather_SDGE_9_row_2000000.csv']

### split the weather data by station

In [14]:
def split_weather_station_by_file (file_weather_data):
    file_chunk_number = int(file_weather_data.split('_row_')[0].split("_")[-1])
    if(file_chunk_number == 0):
        temp_file1_df = pd.read_csv(file_weather_data,header=[0,1])
        temp_file1_df.columns = temp_file1_df.columns.droplevel(1)
    else:
        temp_file1_df = pd.read_csv(file_weather_data)
    
    weather_stid_in_temp_file = temp_file1_df.Station_ID.unique()
    for j in range(len(weather_stid_in_temp_file)):
        temp_weather_stid = weather_stid_in_temp_file[j]
        temp_file_iter = temp_file1_df[temp_file1_df.Station_ID == temp_weather_stid]
        if (temp_weather_stid+".csv" in os.listdir(r"ByWeatherStation/")):
            file_1 = pd.read_csv(r"ByWeatherStation/"+temp_weather_stid+".csv")
            file_1 = pd.concat([file_1,temp_file_iter])
            file_1.to_csv(r"ByWeatherStation/"+temp_weather_stid+".csv",index = False)
        else:
            temp_file_iter.to_csv(r"ByWeatherStation/"+temp_weather_stid+".csv", index = False)

In [15]:
for i in range(len(weather_split_file_list)):
    file_weather_data = weather_split_file_list [i]
    split_weather_station_by_file(file_weather_data)

In [34]:
weather_split_file_list[-1]

'weather_SDGE_9_row_2000000.csv'

## Weather_data calculation by camera name

In [17]:
camera_name_unique_list = camera_station_mapping3.camera_name.unique()
len(camera_station_mapping3.camera_name.unique())

155

### create weather station folder by camera

In [18]:
camera_data_path = weather_data_file_path + r'\by_camera_summary'
os.chdir(weather_data_file_path)
shutil.rmtree(camera_data_path,ignore_errors=True)
os.mkdir("by_camera_summary")

os.chdir(weather_data_file_path + r"\split_files\ByWeatherStation")

In [19]:
###  remove the station id which donot have any weather_data

In [20]:
list_weather_station_data_csv = os.listdir()
weather_station_data_available = []
for i in range (len(list_weather_station_data_csv)):
    temp1= list_weather_station_data_csv[i].split(".")[0]
    weather_station_data_available.append(temp1)

In [21]:
camera_station_mapping3.shape

(463, 11)

In [22]:
camera_station_mapping4 = camera_station_mapping3[camera_station_mapping3['stid'].isin(weather_station_data_available)]
#print(store_df_filtered)

In [23]:
camera_station_mapping4.shape

(463, 11)

In [24]:
camera_station_mapping4.head()

,camera_id,image_id,stid,shortname,distance_m,distance_mi,is_in_direction,rn,Camera_ID,camera_name,distance_mi_inverse
0,hpwren0_unknown direction,tje-1-mobo-c,BFDSD,SDGE,2362.963219,1.468277,NaN,1,hpwren0_unknown direction,tje-1-mobo-c,25.640373
1,hpwren0_unknown direction,tje-1-mobo-c,CVXSD,SDGE,8161.908209,5.071575,NaN,2,hpwren0_unknown direction,tje-1-mobo-c,22.037076
2,hpwren0_unknown direction,tje-1-mobo-c,DJZSD,SDGE,10216.081526,6.347979,NaN,3,hpwren0_unknown direction,tje-1-mobo-c,20.760672
3,hpwren1_north,bm-n-mobo-c,MGDSD,SDGE,4635.802946,2.880554,True,1,hpwren1_north,bm-n-mobo-c,24.228096
4,hpwren1_north,bm-n-mobo-c,MGDSD,SDGE,4635.802946,2.880554,True,1,hpwren1_north,bm-n-mobo,24.228096


### Loop through all weather stations to calculate the per camera summary

In [25]:
for i in range (len(camera_name_unique_list)):
    # Get the weather station related to camera
    camera_name = camera_name_unique_list[i]
    weather_station_of_intrest = list(camera_station_mapping4[camera_station_mapping4["camera_name"] == camera_name ].stid.unique())
    
    #read all the weather station data into single dataframe:
    #print(os.listdir())
    for j in range (len(weather_station_of_intrest)):    
        temp_df = pd.read_csv(weather_station_of_intrest[j] +".csv")
        if (j == 0):
            weather_station_of_intrest_df = temp_df.copy()
        else:
            weather_station_of_intrest_df =  pd.concat([weather_station_of_intrest_df,temp_df])
            
    # Stack all the weather data into single 
    weather_station_of_intrest_df_stacked = pd.melt(weather_station_of_intrest_df, id_vars=['Station_ID', 'Date_Time'],
        var_name='weather_parameter_name', value_name='weather_parameter_value')
    
    # extract key columns from the camera_station mapping file
    camera_station_mapping5 = camera_station_mapping4[camera_station_mapping4.camera_name == camera_name]
    camera_station_mapping5 = camera_station_mapping5 [["camera_name","distance_mi_inverse", "stid"]]
    
    # join the weather data to mapping file.
    weather_station_of_intrest_df_stacked = pd.merge(weather_station_of_intrest_df_stacked, camera_station_mapping5, left_on= "Station_ID", right_on= "stid")
  
    #aggregate the columns to find the sum of all the distance per time/weather_parameter.
    weather_station_of_intrest_df_pivot = pd.pivot_table (weather_station_of_intrest_df_stacked, values="distance_mi_inverse" , 
                                                index= ['Date_Time', 'weather_parameter_name'],
                                                 aggfunc='sum')
    weather_station_of_intrest_df_pivot.reset_index(inplace=True)
    weather_station_of_intrest_df_pivot.rename(columns = {'distance_mi_inverse': 'distance_mi_inverse_sum'}, inplace =True)
    
    
    #Join the weighted sum of distance to stacked columns and calculate the weight factor and normalize the weather data. 
    weather_station_of_intrest_df_stacked_with_weight = pd.merge(weather_station_of_intrest_df_stacked,weather_station_of_intrest_df_pivot, on= ['Date_Time', 'weather_parameter_name'])
    weather_station_of_intrest_df_stacked_with_weight['weather_parameter_value_norm'] =  (
    weather_station_of_intrest_df_stacked_with_weight['weather_parameter_value'] * weather_station_of_intrest_df_stacked_with_weight['distance_mi_inverse'] 
    / weather_station_of_intrest_df_stacked_with_weight['distance_mi_inverse_sum'])
    
    #Round data to nearest 3 digit. (run time is high)
    decimals = 4
    weather_station_of_intrest_df_stacked_with_weight['weather_parameter_value_norm'] = weather_station_of_intrest_df_stacked_with_weight['weather_parameter_value_norm'].apply(lambda x: round(x, decimals))
    
    # Calculate the weighted weather parameter value by per camera + Time. 
    weather_data_per_camera_agg = pd.pivot_table (weather_station_of_intrest_df_stacked_with_weight, values="weather_parameter_value_norm" , 
                                                index= ['Date_Time', 'camera_name'], columns= 'weather_parameter_name',
                                                 aggfunc='sum')
    weather_data_per_camera_agg.reset_index(inplace=True)
    
    weather_data_per_camera_agg_path = camera_data_path + "\\" + camera_name + r'.csv'
    weather_data_per_camera_agg.to_csv(weather_data_per_camera_agg_path,index=False)
    #print (i)                

### Some debug steps

In [26]:
camera_station_mapping3['weather_file_name'] = camera_station_mapping3.stid + ".csv"

In [27]:
available_station_in_mapping = set(camera_station_mapping3.weather_file_name.unique())

camera_station_mapping4.head()

In [28]:
os.chdir(weather_data_file_path + r"\split_files\ByWeatherStation")
available_station_in_file = set(os.listdir())

In [29]:
missing_file = list (available_station_in_mapping - available_station_in_file)

In [30]:
missing_file 

[]

camera_station_mapping4_missing_weather = camera_station_mapping4[camera_station_mapping4['weather_file_name'].isin(missing_file)]
camera_station_mapping4_missing_weather

camera_station_mapping4_missing_weather['properties.description.id'].unique()

In [31]:
camera_station_mapping5 = camera_station_mapping3[~camera_station_mapping3['stid'].isin(weather_station_data_available)]


In [32]:
camera_station_mapping5.head()

,camera_id,image_id,stid,shortname,distance_m,distance_mi,is_in_direction,rn,Camera_ID,camera_name,distance_mi_inverse,weather_file_name


In [33]:
camera_station_mapping5.stid.unique()

array([], dtype=object)